<h1><em>CHEMOSTAT</em> Mass Balance </h1><hr>
<h4> Potential Errors: </h4>
1) Biomass loaded effluent liquid has assumed density of 1000 g/L <br>
2) Acetate as a liquid product not considered <br>
3) Inlet and effluent component fractions are normalized but maybe should not be <br>
4) Liquid draw rate is calculated using an oxygen mass balance, but it may be more appropriate to simply set Draw Rate = Feed Rate <br>

In [26]:
import pandas as pd
from openpyxl import load_workbook
from pprint import pprint
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 500)

<h2>1) Take User Inputs, set File Paths </h2>

In [27]:
run_id     = 'CST' + str(input("Enter the digits from the run ID (CST'XXX'). Only enter the digits!: "))
start_time = float(input("Enter the START time. If timepoint is not in raw data, the next timepoint will be used. "))
end_time   = float(input("Enter the END time. If timepoint is not in raw data, the previous timepoint will be used. "))

infile  = '../CST Data Files/run_set_data_' + run_id + '.csv'
outfile = '../Mass Balance Run List PYTHON v0.5 (new all-in-one script).xlsx'
varfile = 'FDI Tags (do not touch)/tank_tags_map.csv'

Enter the digits from the run ID (CST'XXX'). Only enter the digits!: 322
Enter the START time. If timepoint is not in raw data, the next timepoint will be used. 464
Enter the END time. If timepoint is not in raw data, the previous timepoint will be used. 632


In [28]:
# Determine which gas streams will be considered
used_streams = []

answer = input("Are you using the BLEND gas stream? Enter 'y' or 'n': ")
if answer == 'y':
    used_streams.append('BLEND')
answer = input("Are you using other gas streams? Enter 'y' or 'n': ")
if answer == 'y':
    for stream in ['AIR','CH4','CNG','N2','O2']:
        answer = input(f"Are you using the {stream} stream? Enter 'y' or 'n': ")
        if answer == 'y':
            used_streams.append(stream)

print(f"Gas streams considered in calculations: {used_streams}")

Are you using the BLEND gas stream? Enter 'y' or 'n': n
Are you using other gas streams? Enter 'y' or 'n': y
Are you using the AIR stream? Enter 'y' or 'n': y
Are you using the CH4 stream? Enter 'y' or 'n': y
Are you using the CNG stream? Enter 'y' or 'n': y
Are you using the N2 stream? Enter 'y' or 'n': n
Are you using the O2 stream? Enter 'y' or 'n': n
Gas streams considered in calculations: ['AIR', 'CH4', 'CNG']


<h2>2) Initialize Constants, Assumptions, Variables</h2> <hr>

<h4>Use Python Dictionaries to hold information for components, elements, streams, rates </h4>

Variables starting with _ are constants and will not be changed <br>

Suffixes after the last _ GENERALLY denote units

x: molar/volumetric fraction <br>
w: mass/weight fraction <br>
n: number of moles <br>
F: flow rate <br>

c: component <br>
s: stream <br>
e: element <br>

In [29]:
# Constants, assumptions
_OD_to_gDCW_per_L = 0.300
_T_ref_K = 298.0
_P_ref_bar = 1.0
_R__L_bar_per_K_mol = 0.08314
_working_mass_g = 425.0
_culture_density_g_per_L = 1000.0
_working_vol_L = _working_mass_g / _culture_density_g_per_L
_Biomass_MW = 23.19
_BDO_MW = 90.14


# constants pertaining to elements
_elements = {'C': {'molar_mass': 12.0, 
                   'n_in'      : {'argon':0, 'carbon_dioxide':1, 'ethane':2, 'hydrogen':0, 'methane':1, 'nitrogen':0, 'oxygen':0, 'propane':3},
                   'w_in'      : {'H2O':     0.0000, 
                                  'biomass': 0.5424, 
                                  'BDO':     0.5333}},
             'H': {'molar_mass': 1.0,
                   'n_in'      : {'argon':0, 'carbon_dioxide':0, 'ethane':6, 'hydrogen':2, 'methane':4, 'nitrogen':0, 'oxygen':0, 'propane':8},
                   'w_in'      : {'H2O':     0.1111, 
                                  'biomass': 0.0714, 
                                  'BDO':     0.1111}},
             'O': {'molar_mass': 16.0,
                   'n_in'      : {'argon':0, 'carbon_dioxide':2, 'ethane':0, 'hydrogen':0, 'methane':0, 'nitrogen':0, 'oxygen':2, 'propane':0},
                   'w_in'      : {'H2O':     0.8889, 
                                  'biomass': 0.1778, 
                                  'BDO':     0.3556}},
             'N': {'molar_mass': 14.0,
                   'n_in'      : {'argon':0, 'carbon_dioxide':0, 'ethane':0, 'hydrogen':0, 'methane':0, 'nitrogen':2, 'oxygen':0, 'propane':0},
                   'w_in'      : {'H2O':     0.0000, 
                                  'biomass': 0.2084, 
                                  'BDO':     0.0000}}}
                   
# to be populated with component data and calculations
components = {'argon':         {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'carbon_dioxide':{'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'ethane':        {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'hydrogen':      {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'methane':       {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'nitrogen':      {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'oxygen':        {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None},
              'propane':       {'x_in_raw':       {}, 
                                'x_in_norm':      {}, 
                                'F_in_mph':       {},
                                'F_in_TOTAL_mph': 0.0,
                                    'x_out_raw':      None, 
                                    'x_out_norm':     None, 
                                    'F_out_mph':      None}}

# to be populated with flow rates for every in/out gas stream
# volumetric rates populated from averaged raw data, then used to calculate molar rates using PV=nRT
stream_rates = {'sccm/min':{},
                'mph'     :{}
               }

# Calculated total inlet/effluent rates using NORMALIZED compositions (inlet is NOT simply summed from stream_rates)
total_inlet_norm_mph    = 0.0
total_effluent_norm_mph = 0.0

# to be populated with elemental mass flow rates: gas in/out, liquid in/out, and accumulation, all in g/hr 
elem_rates = {'C':{'inlet_gph'       : 0.0, 
                   'effluent_gph'    : 0.0,
                   'liquid_gph'      : None,
                   'draw_gph'        : None,
                   'accumulation_gph': None},
              'H':{'inlet_gph'       : 0.0, 
                   'effluent_gph'    : 0.0,
                   'liquid_gph'      : None,
                   'draw_gph'        : None,
                   'accumulation_gph': None},
              'O':{'inlet_gph'       : 0.0, 
                   'effluent_gph'    : 0.0,
                   'liquid_gph'      : None,
                   'draw_gph'        : None,
                   'accumulation_gph': None},
              'N':{'inlet_gph'       : 0.0, 
                   'effluent_gph'    : 0.0,
                   'liquid_gph'      : None,
                   'draw_gph'        : None,
                   'accumulation_gph': None}}

# to be populated with elemental closures
closures = {'C': None, 
            'H': None, 
            'O': None, 
            'N': None}

<h2>3) Load Raw Data and Tank-specific Tags</h2>
<h4>Trim data to user input time window, populate tag_dict </h4>

In [30]:
# Load data into DataFrame, and Trim DF to only include data within the specified time window. 
# Set index to ferm age (time), and don't drop column!
# If exact input start_time/end_time don't exist, next/previous timepoints will be picked instead
raw_df = pd.read_csv(infile)
raw_df = raw_df[raw_df['fermentation_age_hours'] >= start_time]
raw_df = raw_df[raw_df['fermentation_age_hours'] <= end_time]
raw_df = raw_df.set_index('fermentation_age_hours', drop=False)

# Remove 'NaN' values from raw data
raw_df.fillna(0.0, inplace=True)
# raw_df = raw_df[raw_df['offgas2.rms_flow'] >= 50.0]

# Pull tank number from raw data as a string, e.g. "02" from "T02"
tank = raw_df.at[raw_df.index[0],'fermentor'][1:3]

# Initialize dict with keys only (values = None)
tag_dict = dict.fromkeys(['Mass spec', 'BDO, mg/L', 'OD, raw', 'Feed Scale, kg', 
                          'BLEND_F_in', 'BLEND_x_argon_in_raw', 'BLEND_x_carbon_dioxide_in_raw', 'BLEND_x_ethane_in_raw', 'BLEND_x_hydrogen_in_raw', 'BLEND_x_methane_in_raw', 'BLEND_x_nitrogen_in_raw', 'BLEND_x_oxygen_in_raw', 'BLEND_x_propane_in_raw',
                          'AIR_F_in',   'AIR_x_argon_in_raw',   'AIR_x_carbon_dioxide_in_raw',   'AIR_x_ethane_in_raw',   'AIR_x_hydrogen_in_raw',   'AIR_x_methane_in_raw',   'AIR_x_nitrogen_in_raw',   'AIR_x_oxygen_in_raw',   'AIR_x_propane_in_raw',
                          'CH4_F_in',   'CH4_x_argon_in_raw',   'CH4_x_carbon_dioxide_in_raw',   'CH4_x_ethane_in_raw',   'CH4_x_hydrogen_in_raw',   'CH4_x_methane_in_raw',   'CH4_x_nitrogen_in_raw',   'CH4_x_oxygen_in_raw',   'CH4_x_propane_in_raw',
                          'CNG_F_in',   'CNG_x_argon_in_raw',   'CNG_x_carbon_dioxide_in_raw',   'CNG_x_ethane_in_raw',   'CNG_x_hydrogen_in_raw',   'CNG_x_methane_in_raw',   'CNG_x_nitrogen_in_raw',   'CNG_x_oxygen_in_raw',   'CNG_x_propane_in_raw',
                          'O2_F_in',    'O2_x_argon_in_raw',    'O2_x_carbon_dioxide_in_raw',    'O2_x_ethane_in_raw',    'O2_x_hydrogen_in_raw',    'O2_x_methane_in_raw',    'O2_x_nitrogen_in_raw',    'O2_x_oxygen_in_raw',    'O2_x_propane_in_raw',
                          'N2_F_in',    'N2_x_argon_in_raw',    'N2_x_carbon_dioxide_in_raw',    'N2_x_ethane_in_raw',    'N2_x_hydrogen_in_raw',    'N2_x_methane_in_raw',    'N2_x_nitrogen_in_raw',    'N2_x_oxygen_in_raw',    'N2_x_propane_in_raw',
                                        'x_argon_out_raw',      'x_carbon_dioxide_out_raw',      'x_ethane_out_raw',      'x_hydrogen_out_raw',      'x_methane_out_raw',      'x_nitrogen_out_raw',      'x_oxygen_out_raw',      'x_propane_out_raw'])

# Fill in dict values with header tags for input tank, from varfile csv
tag_map_df = pd.read_csv(varfile, index_col='Tank')
for key in tag_dict.keys():
    tag_dict[key] = tag_map_df.at[int(tank), key]

start_time = raw_df['fermentation_age_hours'].min()
end_time   = raw_df['fermentation_age_hours'].max()

# Warn if steady state is not reached. Steady state being +/- 25% of BDO, 2% of OD
bdo_start = raw_df.at[start_time, tag_dict['BDO, mg/L']]
bdo_end   = raw_df.at[end_time,   tag_dict['BDO, mg/L']]
od_start  = raw_df.at[start_time, tag_dict['OD, raw']]
od_end    = raw_df.at[end_time,   tag_dict['OD, raw']]

print("-----------------------------------------------------")
print('Run Number:'.ljust(30) + f'{run_id}')
print('Tank:'.ljust(30) + f'T{tank}')
print('Start time:'.ljust(30) + f'{start_time}')
print('End time:'.ljust(30) + f'{end_time}')
print('Initial BDO in mg/L:'.ljust(30) + f'{bdo_start}')
print('Final BDO in mg/L'.ljust(30) + f'{bdo_end}')
print('Initial OD:'.ljust(30) + f'{od_start}')
print('Final OD:'.ljust(30) + f'{od_end}')

-----------------------------------------------------
Run Number:                   CST322
Tank:                         T03
Start time:                   464.0
End time:                     632.0
Initial BDO in mg/L:          8524.5128
Final BDO in mg/L             0.0
Initial OD:                   39.5
Final OD:                     41.7124


<h4>Populate 'components' with inlet and effluent volumetric/molar fractions </h4>

In [31]:
# Average raw data over interval for BDO, OD, and (8 component mass fractions) * (# used + 1 outlet) streams
BDO_mg_per_L = raw_df[tag_dict['BDO, mg/L']].mean()
OD_raw       = raw_df[tag_dict['OD, raw']].mean()

for comp,c in components.items():
    c['x_out_raw'] = raw_df[tag_dict[f'x_{comp}_out_raw']].mean()
    for s in used_streams:
        c['x_in_raw'][s] = raw_df[tag_dict[f'{s}_x_{comp}_in_raw']].mean()

for s in used_streams:
    stream_rates['sccm/min'][s] = raw_df[tag_dict[f'{s}_F_in']].mean()

<h2>4) Mass Balance!</h2>
<h2>4a: Inlet Gas </h2>

In [32]:
# Calculate in_comp_fraction_total
# 8 components * # of used streams - find sum of all component fractions (averaged over time period, given in % points).
# Should be close to 100.0 * # of streams (negligible components are ignored)
in_comp_fraction_total = 0.0
for comp,c in components.items():
    for s in used_streams:
        in_comp_fraction_total += c['x_in_raw'][s]

# With in_comp_fraction_total, calculate normalized compositions so that each of # of streams x 8 component fractions add up to 100 % points
for comp,c in components.items():
    for s in used_streams:
        c['x_in_norm'][s] = (c['x_in_raw'][s]
                             * len(used_streams) 
                             * (100.0 / in_comp_fraction_total)
                            )

# Convert total volumetric flow rate through each used stream (SCCM/min = standard cubic centimeters/min; same as mL/min)
# to total molar flow rates (mph) using PV=nRT.
for s in used_streams:
    stream_rates['mph'][s] = (stream_rates['sccm/min'][s] 
                              * (1/1000) 
                              * 60 
                              * _P_ref_bar 
                              / 
                               (_R__L_bar_per_K_mol * _T_ref_K)
                             )  

# START norm -> raw here!!!    

# Calculate molar flow rates for each comp in each stream = stream flow rate * NORMALIZED component fraction / 100%
# Also sum up to get total inlet rates
for comp,c in components.items():
    for s in used_streams:
        c['F_in_mph'][s] = (stream_rates['mph'][s] 
                            * (c['x_in_raw'][s] / 100.0)
                           )
        
        # sum to get total inlet rate for each component
        c['F_in_TOTAL_mph'] += c['F_in_mph'][s]
        
        # also sum to get total overall inlet rate
        total_inlet_norm_mph += c['F_in_mph'][s]
        
        # Calculate elemental mass flow rate for C H O N
        # = comp molar rate * mol of elem in mol of comp * elem molar mass (g/mol)
        for elem, e in elem_rates.items():
            element = _elements[elem] # brevity
            e['inlet_gph'] += (c['F_in_mph'][s] 
                               * element['n_in'][comp] 
                               * element['molar_mass']
                              )

<h2> 4b: Effluent Gas </h2>

In [33]:
# almost same steps as Inlet Gas, but only one out stream

out_comp_fraction_total = 0.0
for comp,c in components.items():
    out_comp_fraction_total += c['x_out_raw']

for comp,c in components.items():
    c['x_out_norm'] = (c['x_out_raw'] 
                       * (100.0 / out_comp_fraction_total)
                      )

# Need to calculate total effluent molar flow rate using argon mass balance: Argon mph in = Argon mph out
# 1: (total F_in_mph) * (Argon fraction in) = (total F_out_mph) * (Argon fraction out)
# 2: (total F_in_mph) * (Argon fraction in) = (total Argon F_in) 
# (total F_out_mph) = (total Argon F_in) / (Argon fraction out)
argon = components['argon']
total_effluent_norm_mph = (argon['F_in_TOTAL_mph'] 
                           / (argon['x_out_raw'] / 100.0)
                          )

for comp,c in components.items():
    c['F_out_mph'] = (total_effluent_norm_mph 
                      * (c['x_out_raw'] / 100.0)
                     )
    # Calculate elemental mass flow rate for C H O N
    # = comp molar rate * mol of elem in mol of comp * elem molar mass (g/mol)
    for elem, e in elem_rates.items():
        element = _elements[elem] # brevity
        e['effluent_gph'] += (  c['F_out_mph'] 
                              * element['n_in'][comp] 
                              * element['molar_mass'])

<h2> 4c: Liquid Feed </h2>

In [34]:
## Liquid Feed
# Liquid feed rate is d(feed scale)/dt. This mass balance is designed for a single dilution rate (feed rate / V)
t_start       = start_time
t_start_plus1 = start_time + 1.0
t_end         = end_time
t_end_minus1  = end_time - 1.0

# Feed rate = (feed scale reading @ t - feed scale reading @ t+1) / 1 hour
feed_rate_start = raw_df.at[t_start,      tag_dict['Feed Scale, kg']] - raw_df.at[t_start_plus1, tag_dict['Feed Scale, kg']]
feed_rate_end   = raw_df.at[t_end_minus1, tag_dict['Feed Scale, kg']] - raw_df.at[t_end,         tag_dict['Feed Scale, kg']]

# Conditional statement in case we don't have feed scale data, take user input for dilution rate
if feed_rate_end == 0.0:
    dilution_rate     = float(input("Feed scale data is missing. Please enter your assumed dilution rate, e.g. '0.03': "))
    liq_feed_rate_gph = dilution_rate * _working_mass_g
else:
    # scales all should be reading in kg units => convert to g
    liq_feed_rate_gph = feed_rate_end * 1000.00
    dilution_rate     = liq_feed_rate_gph / _working_mass_g


for elem, e in elem_rates.items():
    w_of_elem_in = _elements[elem]['w_in'] # brevity
    
    e['liquid_gph'] = liq_feed_rate_gph * w_of_elem_in['H2O']

<h2> 4d: Accumulation </h2>

In [35]:
# Calculate accumulation (g/L) of liquid products (BDO, BioMass, H2O)

# BDO given in mg/L, convert to g/L
# accumulation of H2O = -(accumulation of BDO and BM)
delta_BDO_gpL =  (bdo_end - bdo_start) / 1000.0
delta_BM_gpL  =  (od_end  - od_start) * _OD_to_gDCW_per_L
delta_H2O_gpL = -(delta_BDO_gpL + delta_BM_gpL)

# Start with Accumulation = Mass * (delta_BDO * sigma_O_BDO + delta_BM * sigma_O_BM + delta_H2O * sigma_O_H2O)
for elem, e in elem_rates.items():
    w_of_elem_in = _elements[elem]['w_in'] # brevity
    
    e['accumulation_gph'] = (_working_vol_L * (  delta_BDO_gpL * w_of_elem_in['BDO']
                                               + delta_BM_gpL  * w_of_elem_in['biomass']
                                               + delta_H2O_gpL * w_of_elem_in['H2O']) 
                             / (end_time - start_time)
                            )

<h2>4e: Liquid Draw </h2>

In [36]:
# Determine average liquid mass fractions of [BDO, BM, H2O] in liquid draw/effluent. Assume 1000.0 g/L density.
w_BDO_gpL = BDO_mg_per_L / 1000
w_BM_gpL  = OD_raw * _OD_to_gDCW_per_L
w_H2O_gpL = (1000.0 
             - w_BDO_gpL 
             - w_BM_gpL)

# Calculate total draw rate in Lph from two oxygen balances:
# 1: O_draw_gph = O_inlet_gph + O_liquid_gph - O_effluent_gph - O_acc 
# 2: O_draw_gph = draw_rate_Lph * (w_BDO_liq * w_O_BDO + w_BM_liq * w_O_BDO + w_H2O_liq * w_O_H2O)
# Another option is to simply set 'Draw Rate, g/hr' = 'Liquid Feed Rate, g/hr'
O = elem_rates['O'] # brevity
w_of_O_in = _elements['O']['w_in'] # brevity

# draw_rate_Lph = ((  O['liquid_gph'] 
#                   + O['inlet_gph'] 
#                   - O['effluent_gph'] 
#                   - O['accumulation_gph']) 
#                  /
#                   (   w_BDO_gpL * w_of_O_in['BDO'] 
#                     + w_BM_gpL  * w_of_O_in['biomass'] 
#                     + w_H2O_gpL * w_of_O_in['H2O']))

draw_rate_Lph = liq_feed_rate_gph / 1000.0

for elem, e in elem_rates.items():
    w_of_elem_in = _elements[elem]['w_in'] # brevity
    
    e['draw_gph'] = (draw_rate_Lph * (  w_BDO_gpL * w_of_elem_in['BDO']
                                      + w_BM_gpL  * w_of_elem_in['biomass']
                                      + w_H2O_gpL * w_of_elem_in['H2O']))

<h2> 5) Elemental Closures </h2>

In [37]:
for elem in closures:
    e = elem_rates[elem]
    closures[elem] = ((  e['effluent_gph'] 
                       + e['draw_gph'] 
                       + e['accumulation_gph'])
                      / 
                       (  e['inlet_gph'] 
                        + e['liquid_gph']))

<h2> 6) KPI Calculations </h2>

In [38]:
KPI_rates_mmph = {'carbon_dioxide':None,
                  'ethane':        None, 
                  'methane':       None,
                  'nitrogen':      None,
                  'oxygen':        None}

for comp, rate in KPI_rates_mmph.items():
    c = components[comp] # brevity
    
    # CER = out - in, XUR = in - out; (1000 mmol/mol) rate conversion
    if comp == 'carbon_dioxide':
        KPI_rates_mmph[comp] = (c['F_out_mph']      - c['F_in_TOTAL_mph']) * 1000.0
    else:
        KPI_rates_mmph[comp] = (c['F_in_TOTAL_mph'] -      c['F_out_mph']) * 1000.0

# Biomass ER: OD * (g DCW / L / OD) * L * (1/hr) / (g DCW / mol DCW) * (1000 mmol DCW / mol DCW) = mmol DCW / hr
biomass_ER = (  OD_raw 
              * _OD_to_gDCW_per_L 
              * _working_vol_L 
              * dilution_rate 
              / _Biomass_MW  
              * 1000)

# 23BDO ER: (mg BDO / L) * L * (1/hr) / (g BDO / mol DCW) * (1000 mmol BDO / mol BDO) * (g BDO / 1000 mg BDO) = mmol BDO / hr
BDO_ER = (BDO_mg_per_L 
          * _working_vol_L 
          * dilution_rate 
          / _BDO_MW)

In [39]:
closures

{'C': 0.9615955099599199,
 'H': 0.924991040300642,
 'O': 0.9190220061109162,
 'N': 0.9925828846013013}

In [40]:
# Stoichiometric C

stoich_C = (KPI_rates_mmph['carbon_dioxide'] + biomass_ER + 4*BDO_ER) / KPI_rates_mmph['methane']
stoich_C

0.8778018397307198

In [41]:
KPI_rates_mmph

{'carbon_dioxide': 22.903792372599124,
 'ethane': 0.47284777076646867,
 'methane': 39.019068066318454,
 'nitrogen': 5.193647641796861,
 'oxygen': 62.03486524697857}

In [42]:
print(biomass_ER)
print(BDO_ER)

7.5126462021000595
0.9586427896233328


In [43]:
print(in_comp_fraction_total)
print(out_comp_fraction_total)

317.1220230311442
99.98401588970438


In [44]:
pprint(components)

{'argon': {'F_in_TOTAL_mph': 0.006060163566362893,
           'F_in_mph': {'AIR': 0.005942105359907431,
                        'CH4': 0.0001155507388343055,
                        'CNG': 2.507467621156335e-06},
           'F_out_mph': 0.006060163566362894,
           'x_in_norm': {'AIR': 0.8642406211971331,
                         'CH4': 0.13462293796182645,
                         'CNG': 0.002921027525770266},
           'x_in_raw': {'AIR': 0.913565780599092,
                        'CH4': 0.14230632810950206,
                        'CNG': 0.003087740527673081},
           'x_out_norm': 0.8158491326683802,
           'x_out_raw': 0.8157187264431687},
 'carbon_dioxide': {'F_in_TOTAL_mph': 0.009051168032959321,
                    'F_in_mph': {'AIR': 0.00020854905100591023,
                                 'CH4': 0.004066171178698211,
                                 'CNG': 0.0047764478032552},
                    'F_out_mph': 0.031954960405558445,
                    'x_in_norm': 

<h2>7) Read output run list as dataframe; Append row; Populate with calculations </h2>

In [45]:
# Read current run list into DataFrame. Append new empty row, and set idx to integer index of this last row
df = pd.read_excel(outfile, index_col=0)
df = df.append(pd.Series(), ignore_index=True)
idx = df.index[-1]

## Choose values/calculations to output to run list spreadsheet

df.at[idx, 'Run_ID'] = run_id
df.at[idx, 'Tank'] = int(tank)

# If exact input start_time/end_time don't exist, next/previous timepoints will be picked instead
df.at[idx, 'Start Time, hours'] = raw_df['fermentation_age_hours'].min()
df.at[idx, 'End Time, hours'] = raw_df['fermentation_age_hours'].max()
df.at[idx, 'Dilution Rate, hrs^-1'] = dilution_rate

df.at[idx, 'Used streams'] = ', '.join(used_streams)

df.at[idx, 'C Closure'] = closures['C']
df.at[idx, 'H Closure'] = closures['H']
df.at[idx, 'O Closure'] = closures['O']
df.at[idx, 'N Closure'] = closures['N']

df.at[idx, 'CER, mmol/hr'] = KPI_rates_mmph['carbon_dioxide']
df.at[idx, 'EUR, mmol/hr'] = KPI_rates_mmph['ethane']
df.at[idx, 'MUR, mmol/hr'] = KPI_rates_mmph['methane']
df.at[idx, 'NUR, mmol/hr'] = KPI_rates_mmph['nitrogen']
df.at[idx, 'OUR, mmol/hr'] = KPI_rates_mmph['oxygen']
df.at[idx, 'Biomass ER, mmol/hr'] = biomass_ER
df.at[idx, '2,3-BDO, mmol/hr'] = BDO_ER

df.at[idx, 'Initial BDO, mg/L'] = bdo_start
df.at[idx, 'Final BDO, mg/L'] = bdo_end
df.at[idx, 'Average BDO, mg/L'] = BDO_mg_per_L

df.at[idx, 'Initial OD, raw'] = od_start
df.at[idx, 'Final OD, raw'] = od_end
df.at[idx, 'Average OD, raw'] = OD_raw

<h4> Add Comment? </h4>

In [46]:
df.at[idx, 'Comments'] = 'offgas2'

<h2> 8) Export trimmed data? Overwrite and Output new list </h2>

In [47]:
# Export raw data?
raw_df.to_csv("raw_df_outputs/trimmed_raw_data_CST322.csv")

In [48]:
df.to_excel(outfile)

<h1> THE END! </h1>

In [49]:
# results.at[idx, 'Total Inlet Gas Rate, mol/hr'] = 

# results.at[idx, 'Inlet C Rate, g/hr'] = 
# results.at[idx, 'Inlet H Rate, g/hr'] = 
# results.at[idx, 'Inlet O Rate, g/hr'] = 
# results.at[idx, 'Inlet N Rate, g/hr'] = 

# results.at[idx, 'Effluent C Rate, g/hr'] = 
# results.at[idx, 'Effluent H Rate, g/hr'] = 
# results.at[idx, 'Effluent O Rate, g/hr'] = 
# results.at[idx, 'Effluent N Rate, g/hr'] = 

# results.at[idx, 'Liquid Feed Rate, g/hr'] = liq_feed_rate_gph

# results.at[idx, 'Liquid C Rate, g/hr'] = 
# results.at[idx, 'Liquid H Rate, g/hr'] = 
# results.at[idx, 'Liquid O Rate, g/hr'] = 
# results.at[idx, 'Liquid N Rate, g/hr'] = 

# results.at[idx, 'Accumulation C, g/hr'] = 
# results.at[idx, 'Accumulation H, g/hr'] = 
# results.at[idx, 'Accumulation O, g/hr'] = 
# results.at[idx, 'Accumulation N, g/hr'] = 

# results.at[idx, 'w_BDO_liq_g/L'] = 
# results.at[idx, 'w_BM_liq_g/L'] = 
# results.at[idx, 'w_H2O_liq_g/L'] = 

# results.at[idx, 'Draw Rate, L/hr'] = 
# results.at[idx, 'Draw Rate, g/hr'] = 

# results.at[idx, 'Draw C Rate, g/hr'] = 
# results.at[idx, 'Draw H Rate, g/hr'] = 
# results.at[idx, 'Draw O Rate, g/hr'] = 
# results.at[idx, 'Draw N Rate, g/hr'] = 

In [25]:
df

,Run_ID,Tank,"Start Time, hours","End Time, hours","Dilution Rate, hrs^-1",Used streams,C Closure,H Closure,O Closure,N Closure,"CER, mmol/hr","EUR, mmol/hr","MUR, mmol/hr","NUR, mmol/hr","OUR, mmol/hr","Biomass ER, mmol/hr","2,3-BDO, mmol/hr",Comments,"Initial BDO, mg/L","Final BDO, mg/L","Average BDO, mg/L","Initial OD, raw","Final OD, raw","Average OD, raw","BLEND_F_in, sccm",BLEND_x_argon_in_raw,BLEND_x_carbon_dioxide_in_raw,BLEND_x_ethane_in_raw,BLEND_x_hydrogen_in_raw,BLEND_x_methane_in_raw,BLEND_x_nitrogen_in_raw,BLEND_x_oxygen_in_raw,BLEND_x_propane_in_raw,"AIR_F_in, sccm",AIR_x_argon_in_raw,AIR_x_carbon_dioxide_in_raw,AIR_x_ethane_in_raw,AIR_x_hydrogen_in_raw,AIR_x_methane_in_raw,AIR_x_nitrogen_in_raw,AIR_x_oxygen_in_raw,AIR_x_propane_in_raw,"CH4_F_in, sccm",CH4_x_argon_in_raw,CH4_x_carbon_dioxide_in_raw,CH4_x_ethane_in_raw,CH4_x_hydrogen_in_raw,CH4_x_methane_in_raw,CH4_x_nitrogen_in_raw,CH4_x_oxygen_in_raw,CH4_x_propane_in_raw,"CNG_F_in, sccm",CNG_x_argon_in_raw,CNG_x_carbon_dioxide_in_raw,CNG_x_ethane_in_raw,CNG_x_hydrogen_in_raw,CNG_x_methane_in_raw,CNG_x_nitrogen_in_raw,CNG_x_oxygen_in_raw,CNG_x_propane_in_raw,"O2_F_in, sccm",O2_x_argon_in_raw,O2_x_carbon_dioxide_in_raw,O2_x_ethane_in_raw,O2_x_hydrogen_in_raw,O2_x_methane_in_raw,O2_x_nitrogen_in_raw,O2_x_oxygen_in_raw,O2_x_propane_in_raw,"N2_F_in, sccm",N2_x_argon_in_raw,N2_x_carbon_dioxide_in_raw,N2_x_ethane_in_raw,N2_x_hydrogen_in_raw,N2_x_methane_in_raw,N2_x_nitrogen_in_raw,N2_x_oxygen_in_raw,N2_x_propane_in_raw,x_argon_out_raw,x_carbon_dioxide_out_raw,x_ethane_out_raw,x_hydrogen_out_raw,x_methane_out_raw,x_nitrogen_out_raw,x_oxygen_out_raw,x_propane_out_raw,AIR_x_argon_in_normalized,CH4_x_argon_in_normalized,CNG_x_argon_in_normalized,N2_x_argon_in_normalized,AIR_x_carbon_dioxide_in_normalized,CH4_x_carbon_dioxide_in_normalized,CNG_x_carbon_dioxide_in_normalized,N2_x_carbon_dioxide_in_normalized,AIR_x_ethane_in_normalized,CH4_x_ethane_in_normalized,CNG_x_ethane_in_normalized,N2_x_ethane_in_normalized,AIR_x_hydrogen_in_normalized,CH4_x_hydrogen_in_normalized,CNG_x_hydrogen_in_normalized,N2_x_hydrogen_in_normalized,AIR_x_methane_in_normalized,CH4_x_methane_in_normalized,CNG_x_methane_in_normalized,N2_x_methane_in_normalized,AIR_x_nitrogen_in_normalized,CH4_x_nitrogen_in_normalized,CNG_x_nitrogen_in_normalized,N2_x_nitrogen_in_normalized,AIR_x_oxygen_in_normalized,CH4_x_oxygen_in_normalized,CNG_x_oxygen_in_normalized,N2_x_oxygen_in_normalized,AIR_x_propane_in_normalized,CH4_x_propane_in_normalized,CNG_x_propane_in_normalized,N2_x_propane_in_normalized,"Inlet Gas MW, g/mol","AIR Inlet Gas Rate, mol/hr","CH4 Inlet Gas Rate, mol/hr","CNG Inlet Gas Rate, mol/hr","N2 Inlet Gas Rate, mol/hr","Total Inlet Gas Rate, mol/hr",AIR_F_argon_in_molph,CH4_F_argon_in_molph,CNG_F_argon_in_molph,N2_F_argon_in_molph,AIR_F_carbon_dioxide_in_molph,CH4_F_carbon_dioxide_in_molph,CNG_F_carbon_dioxide_in_molph,N2_F_carbon_dioxide_in_molph,AIR_F_ethane_in_molph,CH4_F_ethane_in_molph,CNG_F_ethane_in_molph,N2_F_ethane_in_molph,AIR_F_hydrogen_in_molph,CH4_F_hydrogen_in_molph,CNG_F_hydrogen_in_molph,N2_F_hydrogen_in_molph,AIR_F_methane_in_molph,CH4_F_methane_in_molph,CNG_F_methane_in_molph,N2_F_methane_in_molph,AIR_F_nitrogen_in_molph,CH4_F_nitrogen_in_molph,CNG_F_nitrogen_in_molph,N2_F_nitrogen_in_molph,AIR_F_oxygen_in_molph,CH4_F_oxygen_in_molph,CNG_F_oxygen_in_molph,N2_F_oxygen_in_molph,AIR_F_propane_in_molph,CH4_F_propane_in_molph,CNG_F_propane_in_molph,N2_F_propane_in_molph,"Inlet C Rate, mol/hr","Inlet H Rate, mol/hr","Inlet O Rate, mol/hr","Inlet N Rate, mol/hr","Inlet C Rate, g/hr","Inlet H Rate, g/hr","Inlet O Rate, g/hr","Inlet N Rate, g/hr",x_argon_out_normalized,x_carbon_dioxide_out_normalized,x_ethane_out_normalized,x_hydrogen_out_normalized,x_methane_out_normalized,x_nitrogen_out_normalized,x_oxygen_out_normalized,x_propane_out_normalized,"Effluent Gas MW, g/mol","Effluent Gas Rate, mol/hr",F_argon_out_molph,F_carbon_dioxide_out_molph,F_ethane_out_molph,F_hydrogen_out_molph,F_met